## Import libraries:

In [1]:
from datetime import datetime, timedelta
from pathlib import Path
import os
import pandas as pd
import time
import requests
import json
import random

## Define functions:

In [2]:
def IsFilenameAlreadyExisting(filename):
    data_files_existing = os.listdir(directory_actual) + os.listdir(directory_plan)
    if filename in data_files_existing:
        return True
    return False
        
def pauseRandomlyLong():
    delay = round(random.randint(31, 53) / float(17),2)
    time.sleep(delay)

def pauseRandomlyShort():
    delay = round(random.randint(17, 31) / float(17),2)
    time.sleep(delay)
    
def GetIcelandairURL(dest, date_out_str):
    url= f"""https://www.icelandair.com/api/instantSearch/v1/bestPrice/byDay/return/multipleReturnsPerDeparture?
    departure=KEF
    &arrival={dest}
    &locale=is-IS
    &period=0
    &tripDuration=1
    &tripDurationFlexibility=21
    &fromDate={date_out_str}
    &fallbackToRouteCurrency=true
    """
    return url.replace("\n", "").replace(" ", "")

def SampleDataFromPlay(dataFromRequest, samplingList):
    data_home = dataFromRequest.json()['data']['lowestPrices']['homebound']
    data_out = dataFromRequest.json()['data']['lowestPrices']['outbound']

    for out in data_out:
        for home in data_home:
            if home['date'] <= out['date']: continue

            cost = int(home['price']) + int(out['price'])

            samplingList.append(
                {'Airline': 'Play',
                 'C_Date': today_str,
                 'DateOut': out['date'],
                 'DateBack': home['date'],
                 'Destination': dest,
                 'Price': cost})
            
    return samplingList

def SampleDataFromIcelandair(samplingList):
    dates_back = sorted(response.json()['inbound'].keys())

    for date_back_str in dates_back:
        samplingList.append(
        {'Airline': 'Icelandair',
         'C_Date': today_str,
         'DateOut': date_out_str,
         'DateBack': date_back_str,
         'Destination': dest,
         'Price': response.json()['inbound'][date_back_str]['totalFareAmount']})
    
    return samplingList


## Initialize directories:

In [3]:
dir_play = os.path.join('PlaneTicketData', 'Play')
dir_iceair = os.path.join('PlaneTicketData', 'Icelandair')

Path('PlaneTicketData').mkdir(parents=True, exist_ok=True)
Path(dir_play).mkdir(parents=True, exist_ok=True)
Path(dir_iceair).mkdir(parents=True, exist_ok=True)

## Steal prices from Play:

In [4]:
today = datetime.now()
today_str = today.strftime('%Y-%m-%d')

random.seed(today)

numberOfWeeks = 12
dests = ['ALC', 'AMS', 'BCN', 'BER',
         'CDG', 'CPH', 'STN', 'TFS']

target_filename = f"PlaneTicketPrice_Play_{today_str}.csv"
target_filename_path = os.path.join('PlaneTicketData', 'Play') 

if target_filename not in os.listdir(target_filename_path):

    prices = []

    for dest in dests:

        url = "https://flyplay.com/api/graphql"
        data = {
          "operationName": "SearchFormCalendarPrices",
          "variables": {
            "origin": "KEF",
            "destination": dest,
            "startingDate": {
              "month": today.month,
              "year": today.year
            },
            "currency": "ISK",
            "numberOfWeeks": numberOfWeeks,
            "roundTrip": True
          },
          "query": "query SearchFormCalendarPrices($origin: String!, $destination: String!, $startingDate: InputStartingDate!, $currency: String!, $numberOfWeeks: Int, $roundTrip: Boolean!) {\n  lowestPrices(\n    origin: $origin\n    destination: $destination\n    startingDate: $startingDate\n    currency: $currency\n    numberOfWeeks: $numberOfWeeks\n    roundTrip: $roundTrip\n  ) {\n    outbound {\n      date\n      price\n      __typename\n    }\n    homebound {\n      date\n      price\n      __typename\n    }\n    __typename\n  }\n}\n"
        }

        response = requests.post(url, json=data)

        if (response.status_code == 200):
            print(f"Connection to {dest} established.")

            prices = SampleDataFromPlay(response, prices)
            
            print(f"Data sampling for {dest} completed.")

        pauseRandomlyLong()
    
    if prices:
        df = pd.DataFrame(prices)
        df.to_csv(os.path.join(target_filename_path, target_filename), index=False)
        print(f'Data has been saved to file: {target_filename}')
    else:
        print("Data sampling failed.")

else:
    print('File already exists.')

Connection to ALC established.
Data sampling for ALC completed.
Connection to AMS established.
Data sampling for AMS completed.
Connection to BCN established.
Data sampling for BCN completed.
Connection to BER established.
Data sampling for BER completed.
Connection to CDG established.
Data sampling for CDG completed.
Connection to CPH established.
Data sampling for CPH completed.
Connection to STN established.
Data sampling for STN completed.
Connection to TFS established.
Data sampling for TFS completed.
Data has been saved to file: PlaneTicketPrice_Play_2021-10-17.csv


## Steal prices from Icelandair:

In [5]:
today = datetime.now()
today_str = today.strftime('%Y-%m-%d')

number_of_days = 31
random.seed(today)

dests = ['AMS', 'BER', 'BOS', 'ORD', 'DEN',
         'DUB', 'FRA', 'CPH', 'LHR', 'MAN', 
         'MUC', 'NYC', 'MCO', 'OSL', 'CDG', 
         'SEA', 'ARN', 'TFS', 'YTO', 'IAD',]

target_filename = f"PlaneTicketPrice_Icelandair_{today_str}.csv"
target_filename_path = os.path.join('PlaneTicketData', 'Icelandair') 

if target_filename not in os.listdir(target_filename_path):

    startTime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"Data sampling started at {startTime}.")
    
    prices = []

    for dest in dests:
        
        for i in range(1, number_of_days):
            date_out = today + timedelta(days=i)
            date_out_str = date_out.strftime('%Y-%m-%d')

            url = GetIcelandairURL(dest, date_out_str)
            
            response = requests.get(url)
            pauseRandomlyShort()
            
            if (response.status_code == 200):
                
                print(f"{date_out_str}: Connection to {dest} established ({i}/{number_of_days - 1})")
                
                prices = SampleDataFromIcelandair(prices)
            
        print(f"Data sampling for {dest} completed.")
        pauseRandomlyLong()
    
    endTime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"Data sampling finished at {endTime}.")
    if prices:
        df = pd.DataFrame(prices)
        df.to_csv(os.path.join(target_filename_path, target_filename), index=False)
        print(f'Data has been saved to file: {target_filename}')
    else:
        print("No data was sampled in this run.")

else:
    print('File already exists.')

Data sampling started at 2021-10-17 22:20:33.
2021-10-18: Connection to AMS established (2/31)
2021-10-19: Connection to AMS established (3/31)
2021-10-20: Connection to AMS established (4/31)
2021-10-21: Connection to AMS established (5/31)
2021-10-22: Connection to AMS established (6/31)
2021-10-23: Connection to AMS established (7/31)
2021-10-24: Connection to AMS established (8/31)
2021-10-25: Connection to AMS established (9/31)
2021-10-26: Connection to AMS established (10/31)
2021-10-27: Connection to AMS established (11/31)
2021-10-28: Connection to AMS established (12/31)
2021-10-29: Connection to AMS established (13/31)
2021-10-30: Connection to AMS established (14/31)
2021-10-31: Connection to AMS established (15/31)
2021-11-01: Connection to AMS established (16/31)
2021-11-02: Connection to AMS established (17/31)
2021-11-03: Connection to AMS established (18/31)
2021-11-04: Connection to AMS established (19/31)
2021-11-05: Connection to AMS established (20/31)
2021-11-06: 

2021-11-11: Connection to DUB established (26/31)
2021-11-13: Connection to DUB established (28/31)
2021-11-14: Connection to DUB established (29/31)
2021-11-16: Connection to DUB established (31/31)
Data sampling for DUB completed.
2021-10-18: Connection to FRA established (2/31)
2021-10-19: Connection to FRA established (3/31)
2021-10-20: Connection to FRA established (4/31)
2021-10-21: Connection to FRA established (5/31)
2021-10-22: Connection to FRA established (6/31)
2021-10-23: Connection to FRA established (7/31)
2021-10-24: Connection to FRA established (8/31)
2021-10-25: Connection to FRA established (9/31)
2021-10-26: Connection to FRA established (10/31)
2021-10-27: Connection to FRA established (11/31)
2021-10-28: Connection to FRA established (12/31)
2021-10-29: Connection to FRA established (13/31)
2021-10-30: Connection to FRA established (14/31)
2021-10-31: Connection to FRA established (15/31)
2021-11-01: Connection to FRA established (16/31)
2021-11-02: Connection to

2021-11-06: Connection to NYC established (21/31)
2021-11-07: Connection to NYC established (22/31)
2021-11-08: Connection to NYC established (23/31)
2021-11-09: Connection to NYC established (24/31)
2021-11-10: Connection to NYC established (25/31)
2021-11-11: Connection to NYC established (26/31)
2021-11-12: Connection to NYC established (27/31)
2021-11-13: Connection to NYC established (28/31)
2021-11-14: Connection to NYC established (29/31)
2021-11-15: Connection to NYC established (30/31)
2021-11-16: Connection to NYC established (31/31)
Data sampling for NYC completed.
2021-10-18: Connection to MCO established (2/31)
2021-10-21: Connection to MCO established (5/31)
2021-10-22: Connection to MCO established (6/31)
2021-10-25: Connection to MCO established (9/31)
2021-10-29: Connection to MCO established (13/31)
2021-10-30: Connection to MCO established (14/31)
2021-11-01: Connection to MCO established (16/31)
2021-11-04: Connection to MCO established (19/31)
2021-11-05: Connectio

2021-10-25: Connection to YTO established (9/31)
2021-10-27: Connection to YTO established (11/31)
2021-10-28: Connection to YTO established (12/31)
2021-10-29: Connection to YTO established (13/31)
2021-10-30: Connection to YTO established (14/31)
2021-10-31: Connection to YTO established (15/31)
2021-11-01: Connection to YTO established (16/31)
2021-11-02: Connection to YTO established (17/31)
2021-11-03: Connection to YTO established (18/31)
2021-11-04: Connection to YTO established (19/31)
2021-11-05: Connection to YTO established (20/31)
2021-11-06: Connection to YTO established (21/31)
2021-11-07: Connection to YTO established (22/31)
2021-11-08: Connection to YTO established (23/31)
2021-11-09: Connection to YTO established (24/31)
2021-11-10: Connection to YTO established (25/31)
2021-11-11: Connection to YTO established (26/31)
2021-11-12: Connection to YTO established (27/31)
2021-11-13: Connection to YTO established (28/31)
2021-11-14: Connection to YTO established (29/31)
2